And then

- do something with the "double check design matrix" and "first year talk" sections of this notebook
- get working with snakemake

In [ ]:
df = d['df']
tuning_df = sfp.tuning_curves.main(df[df.bootstrap_num.isin(range(2))], bounds=(2**(-5), 2**5))

In [ ]:
sfp.plotting.check_tuning_curves(tuning_df, 'test_%s.svg')

In [ ]:
tuning_df_summary = sfp.tuning_curves.main(ds['df'])
#tuning_df_summary_2 = sfp.tuning_curves.main(ds['df'], bounds=(2**(-4), 2**7))
#tuning_df_summary = pd.concat([tuning_df_summary_1.assign(bounds='tight'), tuning_df_summary_2.assign(bounds='loose')])

In [ ]:
sfp.plotting.check_tuning_curves(tuning_df_summary, 'test_%s.svg')

In [ ]:
df = ds['df']
tmp = df[(df.eccen=='02-03')]

In [ ]:
tmp = tuning_df_summary_1[(tuning_df_summary_1.eccen=='01-02')&(tuning_df_summary_1.stimulus_superclass=='forward diagonal')&(tuning_df_summary_1.varea==1)&(tuning_df_summary_1.frequency_type=='freq_space_distance')]

In [ ]:
g = sns.FacetGrid(tuning_df_summary, col='frequency_type', row='bounds', aspect=2)
g.map(lambda x,y,**kwargs: sns.plt.scatter(np.log2(x), np.log2(y), **kwargs), 'tuning_curve_mu', 'tuning_curve_sigma')

In [ ]:
def plot_tuning_curve(**kwargs):
    data = kwargs.pop('data')
    x, y = sfp.tuning_curves.get_tuning_curve_xy_from_df(data, )#x=np.logspace(-7, -2, 10000, base=2))
    sns.plt.semilogx(x, y, basex=2)

g = sns.FacetGrid(tuning_df_summary[(tuning_df_summary.varea==1)], row='eccen', col='stimulus_superclass', hue='frequency_type')
g.map(sns.plt.scatter, 'frequency_value', 'amplitude_estimate')
g.map_dataframe(plot_tuning_curve)
#for ax in g.axes.flatten():
#    ax.set_xlim((2**-7, 2**-2))
g.add_legend()
g.set_titles("eccen={row_name} | {col_name}")
g.fig.suptitle("varea = 1")
sns.plt.subplots_adjust(top=.95)
g.savefig('test.svg')

In [ ]:
def plot_tuning_curve(**kwargs):
    data = kwargs.pop('data')
    x, y = sfp.tuning_curves.get_tuning_curve_xy_from_df(data)
    sns.plt.semilogx(x, y, basex=2)

g = sns.FacetGrid(tuning_df[(tuning_df.bootstrap_num==1)&(tuning_df.varea==1)], row='eccen', col='stimulus_superclass', hue='frequency_type')
g.map(sns.plt.scatter, 'frequency_value', 'amplitude_estimate')
g.map_dataframe(plot_tuning_curve)

In [ ]:
tmp = tuning_df[(tuning_df.bootstrap_num==2)&(tuning_df.varea==2)&(tuning_df.eccen=='11-12')&(tuning_df.stimulus_superclass=='radial')&(tuning_df.frequency_type=='freq_space_distance')]

In [ ]:
tmp

In [ ]:
sns.plt.scatter(tmp['frequency_value'], tmp['amplitude_estimate'])
plot_tuning_curve(data=tmp)

In [ ]:
a, mode, sigma=(0.98818969370641285, 0.43650524309564304, 0.79026091846015978)
x, y = sfp.tuning_curves.get_tuning_curve_xy(a, mode, sigma)
m, bw, lhm, hhm, warn, x, y = sfp.tuning_curves.log_norm_describe_full(a, mode, sigma)


In [ ]:
a, mode, sigma = (7.6414643191554493e+26, 31.996620943849184, 10.4518910963655)
mu = np.log(mode) + sigma**2
var = (np.exp(sigma**2)-1) * (np.exp(2*mu + sigma**2))

In [ ]:
x, y = sfp.tuning_curves.get_tuning_curve_xy(a, mode, sigma)

In [ ]:
sns.plt.semilogx(x, y, basex=2)
#sns.plt.plot([x.min(), x.max()], [y.max()/2, y.max()/2], 'k--')

In [ ]:
sns.plt.plot(np.log(x), np.log(y))#, basex=2)
#sns.plt.plot([x.min(), x.max()], [y.max()/2, y.max()/2], 'k--')

# Analyzing the first-level results

After running our GLM analysis, we have the estimated amplitude responses of each voxel to each image class. Along with Noah Benson's anatomical template / Bayesian model, we also have each voxel's visual area and location in the visual field (in terms of eccentricity and polar angle). By combining the information contained within them, along with the dataframe describing each stimulus class, we can construct our tuning curves.

In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
import seaborn as sns
%matplotlib inline
import sys
sys.path.append('..')
import sfp
import h5py
import os
import itertools
import pyPyrTools.JBhelpers as jbh
import pyPyrTools as ppt
import scipy as sp
from matplotlib.colors import LinearSegmentedColormap 

/home/billbrod/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/billbrod/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [ ]:
d = sfp.utils.load_data('wlsubj042', 'pilot01', atlas_type='prior', df_mode='full')
ds = sfp.utils.load_data('wlsubj042', 'pilot01', atlas_type='prior', df_mode='summary')

Here we see the different stimulus classes, as plotted in frequency space, colored by their superclass. These numbers are roughly log-spaced (doubling).

We also, in order to double-check some of our calculations, plot the distance and angle in frequency space: they look exactly correct.

In [ ]:
sfp.plotting.stimuli_properties(d['stim_df'])

In [ ]:
sfp.plotting.local_spatial_frequency(d['df'])

In [ ]:
_ = sfp.plotting.stimuli_linear_approximation(d['stim'], d['stim_df'], d['stim_type'], 11, freq_space_distance=181, phi=np.pi/4, stimulus_superclass='radial', save_path='test.svg')

In [ ]:
Rmin, Rmax = sfp.first_level_analysis.find_ecc_range_in_degrees(stim[0,:,:], 12)
print("Inside radius of stimulus annulus: %.02f" % Rmin)
print("Outside radius of stimulus annulus: %.02f" % Rmax)

In [ ]:
tmp_df = tmp_df[tmp_df.eccen=='02-03']

In [ ]:
tmp_df.groupby('freq_space_distance')['amplitude_estimate'].apply(np.percentile, [2.5, 97.5])

In [ ]:
import matplotlib.pyplot as plt
def plot_median(x, y, **kwargs):
    """plot line through center points, for use with seaborn's map_dataframe
    """
    data = kwargs.pop('data')
    plot_data = data.groupby(x)[y].median()
    plt.plot(plot_data.index, plot_data.values, **kwargs)


def scatter_ci_col(x, y, ci, **kwargs):
    """plot center points and specified CIs, for use with seaborn's map_dataframe

    based on seaborn.linearmodels.scatterplot. CIs are taken from a column in this function.
    """
    data = kwargs.pop('data')
    plot_data = data.groupby(x)[y].median()
    plot_cis = data.groupby(x)[ci].median()
    plt.scatter(plot_data.index, plot_data.values, **kwargs)
    for (x, ci), y in zip(plot_cis.iteritems(), plot_data.values):
        plt.plot([x, x], [y+ci, y-ci], **kwargs)


def scatter_ci_dist(x, y, ci_vals=[2.5, 97.5], **kwargs):
    """plot center points and specified CIs, for use with seaborn's map_dataframe

    based on seaborn.linearmodels.scatterplot. CIs are taken from a distribution in this
    function. Therefore, it's assumed that the values being passed to it are values from a
    bootstrap distribution.

    by default, this draws the 95% confidence interval. to change this, change the ci_vals
    argument. for instance, if you only want to draw the median point, pass ci_vals=[50, 50] (this is
    eqiuvalent to just calling plt.scatter)
    """
    data = kwargs.pop('data')
    plot_data = data.groupby(x)[y].median()
    plot_cis = data.groupby(x)[y].apply(np.percentile, ci_vals)
    plt.scatter(plot_data.index, plot_data.values, **kwargs)
    for x, (ci_low, ci_high) in plot_cis.iteritems():
        plt.plot([x, x], [ci_low, ci_high], **kwargs)

def plot_data(df, x_col='freq_space_distance', **kwargs):
    # df = df[df.stimulus_superclass.isin(['radial', 'circular'])]
    col_order = [i for i in sfp.plotting.LOGPOLAR_SUPERCLASS_ORDER if i in df.stimulus_superclass.unique()]
    g = sns.FacetGrid(df, hue='eccen', palette='Reds', size=5, row='varea', col='stimulus_superclass',
                      col_order=col_order)
    if 'amplitude_estimate_std_error' in df.columns:
        g.map_dataframe(plot_median, x_col, 'amplitude_estimate_median')
        g.map_dataframe(scatter_ci_col, x_col, 'amplitude_estimate_median', 'amplitude_estimate_std_error', **kwargs)
    else:
        g.map_dataframe(plot_median, x_col, 'amplitude_estimate')
        g.map_dataframe(scatter_ci_dist, x_col, 'amplitude_estimate', **kwargs)
    for ax in g.axes.flatten():
        ax.set_xscale('log', basex=2)
    g.add_legend()

In [ ]:
plot_data(ds['df'], 'freq_space_distance',)# ci_vals=[16, 84])

In [ ]:
plot_data(d['df'], 'freq_space_distance',)# ci_vals=[16, 84])

In [ ]:
tmp_df = d['df'][d['df'].stimulus_superclass=='circular']
g = sns.FacetGrid(tmp_df, hue='eccen', palette='Reds', size=5,)
#g.map(sns.regplot, 'rounded_freq_space_distance', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'rounded_freq_space_distance', 'amplitude_estimate')
g.map_dataframe(sfp.utils.scatter_ci_dist, 'freq_space_distance', 'amplitude_estimate')
for ax in g.axes.flatten():
    ax.set_xscale('log', basex=2)
g.add_legend()

In [ ]:
g = sns.FacetGrid(df, hue='eccen', palette='Reds', size=5, col='stimulus_superclass', col_wrap=2,
                  col_order=['circular', 'radial', 'forward spiral', 'reverse spiral', 'mixtures'])
#g.map(sns.regplot, 'freq_space_distance', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_distance', 'amplitude_estimate')
g.map_dataframe(sfp.utils.scatter_ci_dist, 'freq_space_distance', 'amplitude_estimate')
for ax in g.axes:
    ax.set_xscale('log', basex=2)
g.add_legend()

If there were no scaling in the visual system, such that neurons at different places in the visual field were expected to have similar properties, the bottom would all line up well, and it doesn't!

In [ ]:
classes_of_interest = []
classes_of_interest.extend(df[(df.stimulus_superclass=='circular')&(df.rounded_freq_space_distance==11)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='radial')&(df.rounded_freq_space_distance==64)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='forward spiral')&(df.rounded_freq_space_distance==23)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='reverse spiral')&(df.rounded_freq_space_distance==23)].stimulus_class.unique())
#classes_of_interest.append(df[df.stimulus_superclass=='mixtures'].stimulus_class.values[0])

stim_idxs = stim_df[stim_df.class_idx.isin(classes_of_interest)].index.values[::8]

jbh.showIm([stim[i, :, :] for i in stim_idxs], ncols=4, zoom=.2)

In [ ]:
# I know this goes from about -pi/2 to pi/2, in pi/12 steps
ticks = [(np.pi*(i-6)/12.) for i in range(13)]
labels = ['$\\frac{%s*\\pi}{12}$'%(i-6) for i in range(13)]

g = sns.FacetGrid(df, hue='eccen', palette='Reds', size=6)
g.map(sns.regplot, 'freq_space_angle', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_angle', 'amplitude_estimate')
#g.map_dataframe(sfp.utils.scatter_ci, 'freq_space_angle', 'amplitude_estimate_median', 'amplitude_estimate_std_error') 
_=g.ax.set_xticks(ticks)
_=g.ax.set_xticklabels(labels)
g.add_legend()

# I know this goes from about -pi/2 to pi/2, in pi/12 steps
ticks = [(np.pi*(i-6)/12.) for i in range(13)]
labels = ['$\\frac{%s*\\pi}{12}$'%(i-6) for i in range(13)]

tmp_df = df[df.stimulus_superclass=='mixtures']
g = sns.FacetGrid(tmp_df, hue='eccen', palette='Reds', size=6)
g.map(sns.regplot, 'freq_space_angle', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_angle', 'amplitude_estimate')
#g.map_dataframe(sfp.utils.scatter_ci, 'freq_space_angle', 'amplitude_estimate_median', 'amplitude_estimate_std_error') 
_=g.ax.set_xticks(ticks)
_=g.ax.set_xticklabels(labels)
g.add_legend()

In [ ]:
angles = sorted(df.freq_space_angle.unique())
stim_idxs = []
for ang in angles:
    class_of_interest = df[(df.freq_space_angle==ang)&(df.rounded_freq_space_distance==32)].stimulus_class.unique()[0]
    stim_idxs.append(stim_df[stim_df.class_idx==class_of_interest].index[0])

#stim_idxs = stim_df[stim_df.class_idx.isin(classes_of_interest)].index.values[::8]

jbh.showIm([stim[i, :, :] for i in stim_idxs], ncols=4, zoom=.2)

In [ ]:
tmp_df = pd.DataFrame(df.groupby(['eccen_bin', 'w_r', 'w_a']).modelmd.mean()).reset_index()

g = sns.FacetGrid(tmp_df, col='eccen_bin', col_wrap=4, size=5)
cbar_ax = g.fig.add_axes([.92, .3, .02, .4])  # <-- Create a colorbar axes
g.map(sfp.utils.scatter_heat, 'w_a', 'w_r', 'amplitude_estimate_median', vmin=tmp_df['amplitude_estimate_median'].min(), 
      vmax=tmp_df['amplitude_estimate_median'].max())
sns.plt.colorbar(cax=cbar_ax)
g.fig.subplots_adjust(right=.9, top=.9)
g.fig.suptitle('Average response amplitude estimates at each point in frequency space')

In [ ]:
tmp_df = pd.pivot_table(df, 'amplitude_estimate_median', 'rounded_freq_space_distance', 'eccen_bin')
norm_df = tmp_df.copy()
for col in norm_df.columns:
    norm_df[col] = norm_df[col] / norm_df[col].max()

In [ ]:
fig = sns.heatmap(tmp_df, cmap='Reds')
fig.invert_yaxis()
fig.set_title('Average response amplitude estimates')

In [ ]:
fig = sns.heatmap(norm_df, cmap='Reds')
fig.invert_yaxis()
fig.set_title('Normalized average response amplitude estimates')

In [ ]:
df = pd.read_csv('/home/billbrod/Data/spatial_frequency_preferences/derivatives/first_level_analysis/stim_class/prior/sub-wlsubj001/ses-pilot01/sub-wlsubj001_ses-pilot01_task-sfp_summary_v1_e2-8_eccen_bin_hemi_bin.csv')

In [ ]:
sns.distplot(df.R2.values)

# Create plots for first year talk

In [ ]:
# for this, we're only using circular results
tmp_df = df[df.stimulus_superclass=='circular']
# tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num', 'hemi']]
tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num']]

In [ ]:
tmper_df = tmp_df[tmp_df.freq_space_distance==6.]
tmper_df = tmper_df.groupby(['eccen', 'bootstrap_num'])['amplitude_estimate'].mean().unstack().reset_index()

In [ ]:
hyp_df = pd.melt(tmp_df, ['eccen', 'amplitude_estimate', 'bootstrap_num'], var_name='Frequency')
hyp_df = pd.DataFrame(hyp_df.groupby(['Frequency', 'value', 'bootstrap_num'])['amplitude_estimate'].mean()).reset_index()

In [ ]:
#print("freq_space_distance min: %.03f, max: %.03f" % (tmp_df.freq_space_distance.min(), tmp_df.freq_space_distance.max()))
#print("Halfway in log space: %.03f" % np.floor(2**((np.log2(181.) + np.log2(6.))/2.)))
# because these are circular, freq_space_distance==w_r
#stim_idx = stim_df[(stim_df.w_r.isin([6, 32, 181]))&(stim_df.w_a==0)].index[::8]
stim_idx = d['stim_df'][(d['stim_df'].w_r.isin([6, 32, ]))&(d['stim_df'].w_a==0)].index[::8]
stims = [d['stim'][i] for i in stim_idx]

We actually want to plot windows of the stimuli instead of just sins

In [ ]:
max_degree_rad = 12
scale_factor = 10
mask = sfp.utils.create_circle_mask(750, 350, scale_factor* 1080/(2*2*max_degree_rad), 1080)
stim_windows = [mask * s + ~mask.astype(bool)*127 for s in stims]


In [ ]:
tmp_df = d['df']
tmp_df = tmp_df[(tmp_df.stimulus_superclass=='circular')&(tmp_df.eccen.isin(['02-03', '03-04', '04-05', '05-06', '06-07', '07-08']))]#&(tmp_df.bootstrap_num.isin(range(5)))]
sfp.plotting.compare_hypotheses(tmp_df)

In [ ]:
tmp_df = d['df']
tmp_df = tmp_df[(tmp_df.bootstrap_num.isin(range(5)))]
sfp.plotting.compare_hypotheses_talk(tmp_df, stims)

In [ ]:
# for this, we're only using circular results
tmp_df = d['df'][d['df'].stimulus_superclass.isin(['circular'])]
# tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num', 'hemi']]
tmp_df = tmp_df[['stimulus_superclass','eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num']]
tmp_df = tmp_df[tmp_df.bootstrap_num.isin(range(5))]

peak_df = sfp.plotting.peak_spatial_frequency(tmp_df, id_vars=['stimulus_superclass'])

In [ ]:
speak_df

In [ ]:
sns.plt.scatter(bandwidth[:,0], bandwidth[:,1], c=sns.color_palette('Reds', 6), s=75)

In [ ]:
ecc = np.linspace(.01, 9, 50)
RF_scale_factor = 4.
V1_RF_size = np.concatenate([np.ones(len(ecc[ecc<.5]))/RF_scale_factor, np.linspace(1/RF_scale_factor, 4/RF_scale_factor, len(ecc[ecc>=.5]))])
#V2_RF_size = np.concatenate([2*np.ones(len(ecc[ecc<4])), np.linspace(2, 2.5, len(ecc[ecc>=4]))])

Olsson_peak = [2.75, 2.11, 1.76, 1.47,1.24, 1.06, .88, .77, .66, .60]
Olsson_ecc = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

with sns.plotting_context('poster', font_scale=1), sns.axes_style('white'):
    # because this doesn't represent data, just intuition, we use this.
#    with plt.xkcd():
    x = np.linspace(.01, 9, 50)
    y = []
    fig, axes = sns.plt.subplots(1,1, figsize=(13,6))
    ax = axes
    # this gives intuitive plots, currently we want the possible hypotheses instead
#         for i in range(3):
#             y.append(10/(x+2)+i)
#             ax.plot(x, y[-1],  label='V%s'%(i+1), color=['r','g','b'][i])
    colors = sns.color_palette(n_colors=3)
    for i, (p, o) in enumerate([[peak_01_s0, opt_a_01_s0], [peak_01, opt_a_01]]):#, [peak_45, opt_a_45]]):
#    ax.plot(ecc, hyperbola(ecc, opt_a), '-', label='scaling')
#        ax.plot(ecc, hyperbola(ecc, o), '-', label='scaling', c= colors[i])
        ax.plot(ecc, hyperbola(ecc, o), '-', label='Subject %s'%(i+1), c= colors[i])
#    ax.plot(ecc, np.ones(len(ecc))*RF_scale_factor, '--', label='constant')
#        ax.set_ylim((0,8))
#    sns.plt.scatter(peak[:, 0], peak[:, 1], c=sns.color_palette('Reds', 6), s=75,)# label='This study')
        if i==1:
            sns.plt.scatter(p[:, 0]+.07, p[:, 1], c=colors[i], s=75,)# label='This study')
        else:
            sns.plt.scatter(p[:, 0], p[:, 1], c=colors[i], s=75,)# label='This study')
#    sns.plt.scatter(Olsson_ecc, Olsson_peak, s=75, c= sns.color_palette('Blues', 10), label='Olsson pilot')
    ax.set_xlabel("Receptive field center eccentricity (degrees)")
    ax.set_ylabel("Peak spatial frequency (cpd)")
    ax.set_title("")
    ax.set_ylim((.5, 4.5))
    ax.set_xlim((0, 11))

    sns.plt.legend(title="Subjects", loc='best')
#    ax.figure.savefig('results-hypotheses.svg', bbox_inches='tight')